In [1]:
import os
os.getcwd()

'/Users/anarpereira/Docs_tcc/TCC'

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split
import os
import random
from PIL import Image

In [3]:
diretorio_base= '/Users/anarpereira/Docs_tcc/TCC/MIND.FungaApp_COMPLETA/'

## Conhecendo os dados

A base de dados utilizada foi criada por pesquisa cidada, conjunto de pesquisadores e amadores, que observam e tiram fotos a fim de alimentar essa base de dados e ajudar os pesquisadores a catalogarem os fungos presentes no Brasil. O projeto possui um guia que ensina como os colaboradores do projeto precisam registrar as fotos, por isso, as fotos possuem boa qualidade, o mesmo fungo é fotografado de vários angulos. Essa fotos sao analisadas por especialistas que confirmam a especie do fungo ou nao, e so depois dessa triagem com os especialistas que as imagens sao adicionadas na base de dados.

A base de dados utilizada possui 509 classes de fungos diferentes, porem 75% da base possui menos de 31 imagens.
Apenas 22 classes possuem mais de 100 imagens, e sao essas 


In [4]:
import os
import pandas as pd

datafiles_to_exclude = []
datafiles_to_stay = []
files_path_to_exclude = []
foldes = []

df = pd.DataFrame(columns=['qty_files', 'folder', 'file_paths'])  # Cria um DataFrame vazio com as colunas desejadas

for pasta in os.listdir(diretorio_base):
    caminho_pasta = os.path.join(diretorio_base, pasta)
    if 'PRE_PROCESSING' not in caminho_pasta:
        if os.path.isdir(caminho_pasta):
            # Obtém os arquivos da pasta
            arquivos = os.listdir(caminho_pasta)

            files_path = [os.path.join(pasta, arquivo) for arquivo in arquivos if (('verde' not in arquivo) 
                                                                                   & ('branco' not in arquivo) 
                                                                                   & ('(1)' not in arquivo) 
                                                                                   & ('cópia' not in arquivo)
                                                                                   & ('copia' not in arquivo)
                                                                                   & ('Copia' not in arquivo)
                                                                               )]

            files_path_to_exclude = [os.path.join(pasta, arquivo) for arquivo in arquivos if (('verde' in arquivo) 
                                                                                   & ('branco' in arquivo) 
                                                                                   & ('(1)' in arquivo) 
                                                                                   & ('cópia' in arquivo)
                                                                                   & ('copia' in arquivo)
                                                                                   & ('Copia' in arquivo)
                                                                                 )]

            # Cria um DataFrame temporário com as informações da iteração atual
            temp_df = pd.DataFrame({'qty_files': len(files_path), 'folder': pasta, 'file_paths': [files_path]}, index=[0])

            # Concatena o DataFrame temporário com o DataFrame principal
            df = pd.concat([df, temp_df], ignore_index=True)

            foldes.append(pasta)
            datafiles_to_stay.extend(files_path)

            datafiles_to_exclude.extend(files_path_to_exclude)

In [5]:
df.shape

(509, 3)

In [13]:
df[['folder','qty_files']].rename(columns={'folder':'Espécie', 'qty_files':'Qtd. Exemplos'}).sort_values(by='Espécie').to_csv('especies_full.csv')

In [13]:
df[['folder','qty_files']].sort_values(by='qty_files', ascending=False).to_csv('especies_full_qty.csv', index=False)

In [14]:
df[['folder','qty_files']]

,folder,qty_files
0,Entolomatricholomatoideum,11
1,Favolusbrasiliensis,131
2,Cladoniarappii,1
3,Cystidiodontiacflaminifera,24
4,SebacinaTremellodendroncfschweinitzii,2
...,...,...
504,Hypotrachynapluriformis,14
505,Hypholomasp,4
506,Trogiacantharelloides,59
507,Panaeolusantillarum,28


In [7]:
# import shutil
# for i in foldes:
#     shutil.rmtree(diretorio_base + i)

In [9]:
for f in os.listdir(diretorio_base):
    r = f.replace(" ","")
    if( r != f):
        os.rename(diretorio_base + f,diretorio_base + r)

In [10]:
for i in os.listdir(diretorio_base):
    if '.DS_Store' not in i:
        for j in os.listdir(diretorio_base + i):
            r = j.replace(" ","")
            if( r != f):
                os.rename(diretorio_base + i + '/' + j, diretorio_base + i + '/' + r)

In [11]:
datafiles_to_exclude_backspace = [i.replace(' ','') for i in datafiles_to_exclude]

In [12]:
for i in datafiles_to_exclude_backspace:
    # Check if the file exists before attempting to delete it
    if os.path.exists(diretorio_base+i):
        os.remove(diretorio_base+i)
        print(f"The file {diretorio_base+i} has been deleted.")
    else:
        print(f"The file {diretorio_base+i} does not exist.")

In [13]:
df.shape

(509, 3)

In [14]:
df['qty_files'] = df['qty_files'].astype(int)

### Pegando as 10 classes que mais possuem imagens, essas serao as classes utilizadas 

In [18]:
df.sort_values(by='qty_files', ascending=False)[:10]

,qty_files,folder,file_paths
356,358,Marasmiusamazonicus,[Marasmiusamazonicus/SSP1118142_11_1_2018_Para...
197,347,Cookeinatricholoma,[Cookeinatricholoma/CCMINDFunga-211211114927Co...
126,326,Phallusindusiatuscomplex,[Phallusindusiatuscomplex/SusanneSourell2014Pa...
394,223,Schizophyllumcommune,[Schizophyllumcommune/FKnc2016-DSCN2298(10)Q(5...
349,198,Coprinellusdisseminatus,[Coprinellusdisseminatus/SSCoprinellus-dissemi...
493,197,Cyathussp,[Cyathussp/SusanneSourell2014Pantanal(1504).jp...
386,193,Oudemansiellacubensis,[Oudemansiellacubensis/00-SSP1152124_15_1_2018...
7,178,Pycnoporussanguineus,"[Pycnoporussanguineus/MIND0117(9).jpg, Pycnopo..."
24,173,Marasmiusgriseoradiatus,[Marasmiusgriseoradiatus/SSMarasmius-sp-BAS-BR...
1,135,Favolusbrasiliensis,[Favolusbrasiliensis/SSPolyporales_Favolus_ten...


In [19]:
df[df['qty_files']>200]

,qty_files,folder,file_paths
126,326,Phallusindusiatuscomplex,[Phallusindusiatuscomplex/SusanneSourell2014Pa...
197,347,Cookeinatricholoma,[Cookeinatricholoma/CCMINDFunga-211211114927Co...
356,358,Marasmiusamazonicus,[Marasmiusamazonicus/SSP1118142_11_1_2018_Para...
394,223,Schizophyllumcommune,[Schizophyllumcommune/FKnc2016-DSCN2298(10)Q(5...


In [20]:
df.head()

,qty_files,folder,file_paths
0,11,Entolomatricholomatoideum,[Entolomatricholomatoideum/LUARSnumeroFK1049(2...
1,135,Favolusbrasiliensis,[Favolusbrasiliensis/SSPolyporales_Favolus_ten...
2,1,Cladoniarappii,[Cladoniarappii/Cladoniarappii_SBUru(653)Emers...
3,24,Cystidiodontiacflaminifera,[Cystidiodontiacflaminifera/MIND0444(21)(quadr...
4,2,SebacinaTremellodendroncfschweinitzii,[SebacinaTremellodendroncfschweinitzii/02-SSTr...


In [22]:
df['qty_files'].describe()

count    509.000000
mean      26.646365
std       39.790701
min        0.000000
25%        6.000000
50%       14.000000
75%       31.000000
max      358.000000
Name: qty_files, dtype: float64

In [23]:
df[df['qty_files']>100]

,qty_files,folder,file_paths
1,135,Favolusbrasiliensis,[Favolusbrasiliensis/SSPolyporales_Favolus_ten...
7,178,Pycnoporussanguineus,"[Pycnoporussanguineus/MIND0117(9).jpg, Pycnopo..."
8,110,Hydropusnigrita,[Hydropusnigrita/SSP1118128_11_1_2018_Paranut_...
24,173,Marasmiusgriseoradiatus,[Marasmiusgriseoradiatus/SSMarasmius-sp-BAS-BR...
58,134,Phillipsiadomingensis,[Phillipsiadomingensis/SSASC_Phillipsia_doming...
64,124,Cookeinaspeciosa,"[Cookeinaspeciosa/FK1838(5)2-WnT2HY.jpg, Cooke..."
126,326,Phallusindusiatuscomplex,[Phallusindusiatuscomplex/SusanneSourell2014Pa...
135,131,Tetrapyrgosalba,[Tetrapyrgosalba/00-SSCampanella-sp-BAS-BR-CJL...
138,134,Auriculariafuscosuccinea,[Auriculariafuscosuccinea/SSP1041628_4_1_2018_...
193,121,Leucocoprinusbrunneoluteus,[Leucocoprinusbrunneoluteus/SSLeucocoprinus_br...


In [24]:
altura_largura = []

In [25]:
datafiles_to_stay_backspace = [i.replace(' ','') for i in datafiles_to_stay]

In [26]:
num_images_per_class = 4

for row_idx, (folder, file_paths) in enumerate(zip(df['folder'], df['file_paths'])):
    count = 0
    fig, axes = plt.subplots(nrows=1, ncols=num_images_per_class, figsize=(15, 10))

    for col_idx, file_path in enumerate(file_paths):
        try:
            if count < num_images_per_class:
                img_path = os.path.join(diretorio_base, file_path)
                imagem = mpimg.imread(img_path)
                axes[col_idx].imshow(imagem)
                count += 1
            else:
                plt.savefig(f'figure_{folder}.jpg', format='jpg')
        except IndexError:
            axes[col_idx].axis('off')  # Desativa o eixo se não houver imagem
            axes[col_idx].set_frame_on(False)  # Remove o quadro em torno do subplot
            print('IndexError', row_idx, col_idx)
        except IsADirectoryError:
            print('IsADirectoryError')
        
    # Set the class label as the title for the first image in the row

plt.tight_layout()
plt.show()


KeyboardInterrupt



Error in callback <function flush_figures at 0x7f84747bf760> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt



In [ ]:
files = []
foldes = []

df = pd.DataFrame(columns=['qty_files', 'folder', 'file_paths'])  # Cria um DataFrame vazio com as colunas desejadas

for pasta in os.listdir(diretorio_base):
    caminho_pasta = os.path.join(diretorio_base, pasta)
    if os.path.isdir(caminho_pasta):
        # Obtém os arquivos da pasta
        arquivos = os.listdir(caminho_pasta)
        
        files_path = [os.path.join(pasta, arquivo) for arquivo in arquivos]

        # Cria um DataFrame temporário com as informações da iteração atual
        temp_df = pd.DataFrame({'qty_files': len(files_path), 'folder': pasta, 'file_paths': [files_path]}, index=[0])
        
        # Concatena o DataFrame temporário com o DataFrame principal
        df = pd.concat([df, temp_df], ignore_index=True)
        files.extend(files_path)

In [ ]:
df.shape

In [51]:
df['qty_files'].astype(int).describe()

count    509.000000
mean      35.385069
std       54.193996
min        1.000000
25%        8.000000
50%       20.000000
75%       41.000000
max      714.000000
Name: qty_files, dtype: float64

In [52]:
df[df['qty_files']>100].shape

(33, 3)

### Dimensoes das imagens

Todas as imagens do dataset tem 1000px por 1000

Vamos redimensionar para 224x224

In [35]:
files_out_default = []
for caminho_completo in files:
    if '.ipynb' not in caminho_completo:
        with open(diretorio_base + caminho_completo, 'rb') as arquivo_imagem:
            altura, largura = Image.open(diretorio_base + caminho_completo).size
            if altura != 1000 or largura != 1000:
                files_out_default.append(diretorio_base + caminho_completo)

In [36]:
files_out_default

[]

In [39]:
for pasta in os.listdir(diretorio_base):
    if '.ipynb' not in pasta and '.DS_' not in pasta:
        if os.path.isdir(diretorio_base + pasta):
            new_directory = os.path.join(diretorio_base, pasta + '_PRE_PROCESSING')
            if not os.path.exists(new_directory):
                os.makedirs(new_directory, exist_ok=True) 
                
                for file, count in zip(os.listdir(diretorio_base + pasta), range(0, len(os.listdir(diretorio_base+pasta)))):
                    if '.jpg' in file:
                        img = Image.open(os.path.join(diretorio_base, pasta, file))
                        resized = img.resize((224, 224))
                        resized.save(os.path.join(new_directory, pasta.lower() + f'_{count}.jpg'))        